In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv("survey_results_public.csv")
data.columns

Index(['ResponseId', 'MainBranch', 'Employment', 'RemoteWork',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       'LearnCodeCoursesCert', 'YearsCode', 'YearsCodePro', 'DevType',
       'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'Country', 'Currency',
       'CompTotal', 'CompFreq', 'LanguageHaveWorkedWith',
       'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith',
       'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith',
       'PlatformWantToWorkWith', 'WebframeHaveWorkedWith',
       'WebframeWantToWorkWith', 'MiscTechHaveWorkedWith',
       'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith',
       'ToolsTechWantToWorkWith', 'NEWCollabToolsHaveWorkedWith',
       'NEWCollabToolsWantToWorkWith', 'OpSysProfessional use',
       'OpSysPersonal use', 'VersionControlSystem', 'VCInteraction',
       'VCHostingPersonal use', 'VCHostingProfessional use',
       'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith',
       'OfficeStackSyncHaveWork

In [3]:
cols_to_keep = ['MainBranch', 'Employment', 'RemoteWork', 'YearsCode', 'CompTotal', 'CompFreq', 'LanguageHaveWorkedWith', 'Country']

In [4]:
data = data[cols_to_keep]

In [5]:
data = data[~data["MainBranch"].isin(["None of these", "I used to be a developer by profession, but no longer am"])] # remove non-developers

In [6]:
# transform years to code
transformation_dict = {
    "Less than 1 year": 0.0,
    "More than 50 years": 51
}

def years_code_to_float(data):
    try:
        return float(data)
    except:
        return transformation_dict[data]

In [7]:
data["YearsCode"] = data["YearsCode"].apply(years_code_to_float) # convert YearsCode to float

In [8]:
data = data[data["CompTotal"] < 1e+7] # remove rows with too large salary

In [9]:
languages = data["LanguageHaveWorkedWith"]

In [10]:
def apply_split(row):
    try:
        return row.split(";")
    except:
        return [row]

In [11]:
languages = languages.apply(apply_split)

In [12]:
unique = np.unique(np.concatenate(languages.values)) # get all the unique languages

In [13]:
# create columns for each language and mark if user have worked with it

for unique_lang in unique:
    data[unique_lang] = data["LanguageHaveWorkedWith"].apply(apply_split)

for unique_lang in unique:
    data[unique_lang] = data[unique_lang].apply(lambda r: unique_lang in r)

In [14]:
data

,MainBranch,Employment,RemoteWork,YearsCode,CompTotal,CompFreq,LanguageHaveWorkedWith,Country,APL,Assembly,...,Ruby,Rust,SAS,SQL,Scala,Solidity,Swift,TypeScript,VBA,nan
2,"I am not primarily a developer, but I write co...","Employed, full-time","Hybrid (some remote, some in-person)",14.0,32000.0,Yearly,C#;C++;HTML/CSS;JavaScript;Python,United Kingdom of Great Britain and Northern I...,False,False,...,False,False,False,False,False,False,False,False,False,False
3,I am a developer by profession,"Employed, full-time",Fully remote,20.0,60000.0,Monthly,C#;JavaScript;SQL;TypeScript,Israel,False,False,...,False,False,False,True,False,False,False,True,False,False
8,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",6.0,46000.0,Yearly,NaN,Netherlands,False,False,...,False,False,False,False,False,False,False,False,False,False
10,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",5.0,48000.0,Yearly,Bash/Shell;C#;HTML/CSS;JavaScript;PowerShell;SQL,United Kingdom of Great Britain and Northern I...,False,False,...,False,False,False,True,False,False,False,False,False,False
11,"I am not primarily a developer, but I write co...","Employed, full-time;Independent contractor, fr...",Fully remote,12.0,194400.0,Yearly,C#;HTML/CSS;JavaScript;PowerShell;Python;Rust;SQL,United States of America,False,False,...,False,True,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73259,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",6.0,8000.0,Monthly,Elixir,Poland,False,False,...,False,False,False,False,False,False,False,False,False,False
73261,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",9.0,36000.0,Yearly,Bash/Shell;HTML/CSS;Java;JavaScript;Python;SQL...,France,False,False,...,False,False,False,True,False,False,False,True,False,False
73263,I am a developer by profession,"Employed, full-time",Fully remote,8.0,60000.0,Yearly,Bash/Shell;Dart;JavaScript;PHP;Python;SQL;Type...,Nigeria,False,False,...,False,False,False,True,False,False,False,True,False,False
73264,I am a developer by profession,"Employed, full-time",Full in-person,6.0,107000.0,Yearly,Bash/Shell;HTML/CSS;JavaScript;Python;SQL,United States of America,False,False,...,False,False,False,True,False,False,False,False,False,False


In [15]:
cross_df = pd.crosstab(data.index, data["RemoteWork"])

In [16]:
cross_df

RemoteWork,Full in-person,Fully remote,"Hybrid (some remote, some in-person)"
row_0,,,
2,0,0,1
3,0,1,0
8,0,0,1
10,0,0,1
11,0,1,0
...,...,...,...
73259,0,0,1
73261,0,0,1
73263,0,1,0


In [17]:
data = pd.concat([data, cross_df], axis=1)

In [18]:
data["MainBranch"].value_counts()

I am a developer by profession                                                   34730
I am not primarily a developer, but I write code sometimes as part of my work     3011
Name: MainBranch, dtype: int64

In [19]:
countries_to_keep = pd.DataFrame(data["Country"].value_counts() > 100).reset_index() # countries with 100 or more survey rows

In [20]:
countries_to_keep = countries_to_keep.drop(
    countries_to_keep[countries_to_keep["Country"] == False].index
    ).drop(columns=["Country"])["index"]

In [21]:
countries_to_keep

0                              United States of America
1                                               Germany
2     United Kingdom of Great Britain and Northern I...
3                                                 India
4                                                Canada
5                                                France
6                                                Brazil
7                                                 Spain
8                                                Poland
9                                           Netherlands
10                                            Australia
11                                                Italy
12                                               Sweden
13                                   Russian Federation
14                                          Switzerland
15                                               Turkey
16                                              Austria
17                                              

In [22]:
data = data.drop(data[~data["Country"].isin(countries_to_keep)].index) # keep only countries with 100 or more rows

In [23]:
cross_df = pd.crosstab(data.index, [data["Country"]])

In [24]:
cross_df

Country,Argentina,Australia,Austria,Bangladesh,Belgium,Brazil,Bulgaria,Canada,Chile,China,...,South Africa,Spain,Sri Lanka,Sweden,Switzerland,Taiwan,Turkey,Ukraine,United Kingdom of Great Britain and Northern Ireland,United States of America
row_0,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73259,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73263,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
data = pd.concat([data, cross_df], axis=1)

In [26]:
data

,MainBranch,Employment,RemoteWork,YearsCode,CompTotal,CompFreq,LanguageHaveWorkedWith,Country,APL,Assembly,...,South Africa,Spain,Sri Lanka,Sweden,Switzerland,Taiwan,Turkey,Ukraine,United Kingdom of Great Britain and Northern Ireland,United States of America
2,"I am not primarily a developer, but I write co...","Employed, full-time","Hybrid (some remote, some in-person)",14.0,32000.0,Yearly,C#;C++;HTML/CSS;JavaScript;Python,United Kingdom of Great Britain and Northern I...,False,False,...,0,0,0,0,0,0,0,0,1,0
3,I am a developer by profession,"Employed, full-time",Fully remote,20.0,60000.0,Monthly,C#;JavaScript;SQL;TypeScript,Israel,False,False,...,0,0,0,0,0,0,0,0,0,0
8,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",6.0,46000.0,Yearly,NaN,Netherlands,False,False,...,0,0,0,0,0,0,0,0,0,0
10,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",5.0,48000.0,Yearly,Bash/Shell;C#;HTML/CSS;JavaScript;PowerShell;SQL,United Kingdom of Great Britain and Northern I...,False,False,...,0,0,0,0,0,0,0,0,1,0
11,"I am not primarily a developer, but I write co...","Employed, full-time;Independent contractor, fr...",Fully remote,12.0,194400.0,Yearly,C#;HTML/CSS;JavaScript;PowerShell;Python;Rust;SQL,United States of America,False,False,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73259,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",6.0,8000.0,Monthly,Elixir,Poland,False,False,...,0,0,0,0,0,0,0,0,0,0
73261,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",9.0,36000.0,Yearly,Bash/Shell;HTML/CSS;Java;JavaScript;Python;SQL...,France,False,False,...,0,0,0,0,0,0,0,0,0,0
73263,I am a developer by profession,"Employed, full-time",Fully remote,8.0,60000.0,Yearly,Bash/Shell;Dart;JavaScript;PHP;Python;SQL;Type...,Nigeria,False,False,...,0,0,0,0,0,0,0,0,0,0
73264,I am a developer by profession,"Employed, full-time",Full in-person,6.0,107000.0,Yearly,Bash/Shell;HTML/CSS;JavaScript;Python;SQL,United States of America,False,False,...,0,0,0,0,0,0,0,0,0,1


In [27]:
data["DeveloperByProfession"] = data["MainBranch"] == "I am a developer by profession"

In [28]:
data

,MainBranch,Employment,RemoteWork,YearsCode,CompTotal,CompFreq,LanguageHaveWorkedWith,Country,APL,Assembly,...,Spain,Sri Lanka,Sweden,Switzerland,Taiwan,Turkey,Ukraine,United Kingdom of Great Britain and Northern Ireland,United States of America,DeveloperByProfession
2,"I am not primarily a developer, but I write co...","Employed, full-time","Hybrid (some remote, some in-person)",14.0,32000.0,Yearly,C#;C++;HTML/CSS;JavaScript;Python,United Kingdom of Great Britain and Northern I...,False,False,...,0,0,0,0,0,0,0,1,0,False
3,I am a developer by profession,"Employed, full-time",Fully remote,20.0,60000.0,Monthly,C#;JavaScript;SQL;TypeScript,Israel,False,False,...,0,0,0,0,0,0,0,0,0,True
8,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",6.0,46000.0,Yearly,NaN,Netherlands,False,False,...,0,0,0,0,0,0,0,0,0,True
10,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",5.0,48000.0,Yearly,Bash/Shell;C#;HTML/CSS;JavaScript;PowerShell;SQL,United Kingdom of Great Britain and Northern I...,False,False,...,0,0,0,0,0,0,0,1,0,True
11,"I am not primarily a developer, but I write co...","Employed, full-time;Independent contractor, fr...",Fully remote,12.0,194400.0,Yearly,C#;HTML/CSS;JavaScript;PowerShell;Python;Rust;SQL,United States of America,False,False,...,0,0,0,0,0,0,0,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73259,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",6.0,8000.0,Monthly,Elixir,Poland,False,False,...,0,0,0,0,0,0,0,0,0,True
73261,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",9.0,36000.0,Yearly,Bash/Shell;HTML/CSS;Java;JavaScript;Python;SQL...,France,False,False,...,0,0,0,0,0,0,0,0,0,True
73263,I am a developer by profession,"Employed, full-time",Fully remote,8.0,60000.0,Yearly,Bash/Shell;Dart;JavaScript;PHP;Python;SQL;Type...,Nigeria,False,False,...,0,0,0,0,0,0,0,0,0,True
73264,I am a developer by profession,"Employed, full-time",Full in-person,6.0,107000.0,Yearly,Bash/Shell;HTML/CSS;JavaScript;Python;SQL,United States of America,False,False,...,0,0,0,0,0,0,0,0,1,True


In [29]:
remove_columns = ["MainBranch", "Employment", "RemoteWork", "CompFreq", "LanguageHaveWorkedWith", "Country", "nan"]

In [30]:
data.drop(columns=remove_columns, inplace=True)

In [31]:
data.astype('float32').to_csv("learn/surveys.csv")